<a href="https://colab.research.google.com/github/ggyppsyy/colab_experiments/blob/master/Copy_of_MaskGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import math
!pip install dropbox

import IPython.display as display

import dropbox
import os
import random
import time

import skimage
from PIL import Image
from IPython.display import display
import multiprocessing as mp


import tensorflow as tf
from tensorflow.keras import layers
tf.enable_eager_execution()
assert tf.multiply(6, 7).numpy() == 42

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

import time

current_milli_time = lambda: str(int(round(time.time())))

dbx = dropbox.Dropbox('xxx')

dbx.users_get_current_account()

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
path = '/art/paintings/round1/'
response = dbx.files_list_folder(path=path)
art_entries = []
for entry in response.entries:
  art_entries.append(entry.path_lower)
art_image_count = len(art_entries)
if not os.path.exists(path):
    os.makedirs(path)
for p in art_entries:
    md, res = dbx.files_download(p)
    with open(p, 'wb+') as f:
        f.write(res.content)
art_entries

['/art/paintings/round1/a-glass-1911.jpg!large.jpg',
 '/art/paintings/round1/bread-and-dish-with-fruits-on-the-table.jpg!large.jpg',
 '/art/paintings/round1/clarinet-1911.jpg!large.jpg',
 '/art/paintings/round1/clarinetist.jpg!large.jpg',
 '/art/paintings/round1/guitar-player-1910.jpg!large.jpg',
 '/art/paintings/round1/friendship-1908.jpg!large.jpg',
 '/art/paintings/round1/dance-of-the-veils-1907.jpg!large.jpg',
 '/art/paintings/round1/girl-with-mandolin-fanny-tellier-1910.jpg!large.jpg',
 '/art/paintings/round1/man-with-a-guitar-1911-1.jpg!large.jpg',
 '/art/paintings/round1/harlequinesque-personage-1913.jpg!large.jpg',
 '/art/paintings/round1/harlequin-leaning-1909.jpg',
 '/art/paintings/round1/houses-on-the-hill-1909.jpg!large.jpg',
 '/art/paintings/round1/man-with-guitar-1912.jpg!large.jpg',
 '/art/paintings/round1/mandolin-1914.jpg!large.jpg',
 '/art/paintings/round1/my-beautiful-woman-with-guitar-1912.jpg!large.jpg',
 '/art/paintings/round1/nude-figure.jpg!large.jpg',
 '/art/pa

In [3]:
import pathlib
data_root_orig = tf.keras.utils.get_file('flower_photos',
                                         'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz', 
                                         untar=True)
data_root = pathlib.Path(data_root_orig)
print(data_root)

/root/.keras/datasets/flower_photos


In [4]:
flower_image_paths = list(data_root.glob('*/*'))
flower_image_paths = [str(path) for path in flower_image_paths]
random.shuffle(flower_image_paths)

flower_image_count = len(flower_image_paths)
flower_image_count

3670

In [0]:
class GeneratorModel(tf.keras.Model):
    def __init__(self):
      super(GeneratorModel, self).__init__(name='GeneratorModel')
      filters = [32,48,48,64]
      self.defilters = np.array([64,48,32,3])
      kernel_sizes = [[3,3],[3,3],[3,3],[3,3]]
      self.cnn1 = layers.Conv2D(filters=filters[0],
                                kernel_size=kernel_sizes[0],
                                activation=None,
                                padding='same')
      self.norm1 = layers.BatchNormalization()
      self.pool1 = layers.MaxPool2D(padding='same')
      self.cnn2 = layers.Conv2D(filters=filters[1],
                                kernel_size=kernel_sizes[1],
                                strides=1,
                                activation=None,
                                padding='same')
      self.norm2 = layers.BatchNormalization()
      self.pool2 = layers.MaxPool2D(padding='same')
      self.cnn3 = layers.Conv2D(filters=filters[2],
                                kernel_size=kernel_sizes[2],
                                strides=1,
                                activation=None,
                                padding='same')
      self.norm3 = layers.BatchNormalization()
      self.pool3 = layers.MaxPool2D(padding='same')
      self.cnn4 = layers.Conv2D(filters=filters[3],
                                kernel_size=kernel_sizes[3],
                                strides=1,
                                activation=None,
                                padding='same')
      self.norm4 = layers.BatchNormalization()
      self.pool4 = layers.MaxPool2D(padding='same')
      self.cnn5 = layers.Conv2D(filters=filters[3],
                                kernel_size=kernel_sizes[3],
                                strides=1,
                                activation=None,
                                padding='same')
      self.norm5 = layers.BatchNormalization()
      self.pool5 = layers.MaxPool2D(padding='same')
      self.cnn6 = layers.Conv2D(filters=filters[3],
                                kernel_size=kernel_sizes[3],
                                strides=1,
                                activation=None,
                                padding='same')
      self.norm7 = layers.BatchNormalization()
      self.upsample1 = layers.UpSampling2D()
      self.trans1 = layers.Conv2D(filters=self.defilters[0], 
                                           kernel_size=kernel_sizes[0],
                                           strides = 1,
                                           padding='same')
      self.upsample1_1 = layers.UpSampling2D()
      self.trans1_1 = layers.Conv2D(filters=self.defilters[0], 
                                           kernel_size=kernel_sizes[0],
                                           strides = 1,
                                           padding='same')
      self.norm5 = layers.BatchNormalization() 
      self.upsample2 = layers.UpSampling2D()
      self.trans2 = layers.Conv2D(filters=self.defilters[1],
                                           kernel_size=kernel_sizes[1],
                                           strides = 1,
                                           padding='same')
      self.upsample2_1 = layers.UpSampling2D()
      self.trans2_1 = layers.Conv2D(filters=self.defilters[0], 
                                           kernel_size=kernel_sizes[0],
                                           strides = 1,
                                           padding='same')
      self.norm6 = layers.BatchNormalization() 
      self.upsample3 = layers.UpSampling2D()
      self.trans3 = layers.Conv2D(filters=self.defilters[2],
                                           kernel_size=kernel_sizes[2],
                                           strides = 1,
                                           padding='same')
      self.trans3_1 = layers.Conv2D(filters=self.defilters[0], 
                                           kernel_size=kernel_sizes[0],
                                           strides = 1,
                                           padding='same')
      self.norm7 = layers.BatchNormalization() 
      self.trans4 = layers.Conv2D(filters=self.defilters[3],
                                           kernel_size=kernel_sizes[3],
                                           strides = 1,
                                           padding='same')
      self.trans4_1 = layers.Conv2D(filters=self.defilters[0], 
                                           kernel_size=kernel_sizes[0],
                                           strides = 1,
                                           padding='same')
      self.norm8 = layers.BatchNormalization() 
      self.final_conv = layers.Conv2D(filters=self.defilters[3],
                                  kernel_size=[1,1],
                                  padding='same')
      
      
    def call(self, inputs, training=False):
      x = self.cnn1(inputs)
      if (training):
          x = self.norm1(x)
      x1 = tf.keras.activations.selu(x)
      x = self.cnn2(self.pool1(x1))
      if (training):
          x = self.norm2(x)
      x2 = tf.keras.activations.selu(x)
      x = self.cnn3(self.pool2(x2))
      if (training):
          x = self.norm3(x)
      x3 = tf.keras.activations.selu(x)
      x = self.cnn4(self.pool3(x3))
      if (training):
          x = self.norm4(x)
      x = tf.keras.activations.tanh(x)
      x = self.cnn5(self.pool4(x))
      if (training):
          x = self.norm5(x)
      x = tf.keras.activations.tanh(x)
      x = self.cnn6(self.pool5(x))
      if (training):
          x = self.norm6(x)
      x = tf.keras.activations.tanh(x)
      x = self.upsample1(x)
      x = self.trans1(x)
      x = tf.keras.activations.selu(x)
      x = self.upsample1_1(x)
      x = self.trans1_1(x)
      if (training):
          x = self.norm5(x)
      x = tf.keras.activations.selu(x)
      x = self.upsample2(x)
      x = tf.concat([x, x3],axis=-1)
      x = self.trans2(x)
      x = tf.keras.activations.selu(x)
      x = self.upsample2_1(x)
      x = self.trans2_1(x)
      if (training):
          x = self.norm6(x)
      x = tf.keras.activations.selu(x)
      x = tf.concat([x, x2],axis=-1)
      x = self.upsample3(x)
      x = self.trans3(x)
      x = self.trans3_1(x)
      if (training):
          x = self.norm7(x)
      x = tf.keras.activations.selu(x)
      x = tf.concat([x, x1],axis=-1)
      x = self.trans4(x)
      x = self.trans4_1(x)
      if (training):
          x = self.norm8(x)
      x = tf.keras.activations.selu(x)
      x = tf.concat([x, inputs],axis=-1)
      x = tf.keras.activations.selu(self.final_conv(x))
      return x

In [0]:
class DiscriminatorModel(tf.keras.Model):

    def __init__(self):
        super(DiscriminatorModel, self).__init__(name='DiscriminatorModel')
        filters = [64,64,64,64,64,32,16]
        kernel_sizes = [[3,3],[3,3],[3,3],[3,3],[3,3],[3,3],[3,3]]
        self.cnn1 = layers.Conv2D(filters=filters[0],
                                  kernel_size=kernel_sizes[0],
                                  activation=None)
        self.norm1 = layers.BatchNormalization()
        self.pool1 = layers.MaxPool2D(padding='same')
        self.cnn2 = layers.Conv2D(filters=filters[1],
                                  kernel_size=kernel_sizes[1],
                                  activation=None)
        self.norm2 = layers.BatchNormalization()
        #self.pool = layers.MaxPool2D(padding='same')
        self.cnn3 = layers.Conv2D(filters=filters[2],
                                  kernel_size=kernel_sizes[2],
                                  activation=None)
        self.norm3 = layers.BatchNormalization()
        self.pool2 = layers.MaxPool2D(padding='same')
        self.cnn4 = layers.Conv2D(filters=filters[3],
                                  kernel_size=kernel_sizes[3],
                                  activation=None)
        self.norm4 = layers.BatchNormalization()
        #self.pool = layers.MaxPool2D(padding='same')
        self.cnn5 = layers.Conv2D(filters=filters[4],
                                  kernel_size=kernel_sizes[4],
                                  activation=None)
        self.norm5 = layers.BatchNormalization()
        self.pool3 = layers.MaxPool2D(padding='same')
        self.cnn6 = layers.Conv2D(filters=filters[5],
                                   kernel_size=kernel_sizes[5],
                                   activation=None)
        self.norm6 = layers.BatchNormalization()
        self.pool4 = layers.MaxPool2D(padding='same')
        self.cnn7 = layers.Conv2D(filters=filters[6],
                                  kernel_size=kernel_sizes[6],
                                  activation=None)
        self.norm7= layers.BatchNormalization()
        self.flatten = layers.Flatten()
        self.final_dense = layers.Dense(1,activation='sigmoid')
      
    def call(self, inputs, training=False):
        x = tf.keras.activations.selu(self.norm1(self.cnn1(inputs)))
        x = tf.keras.activations.selu(self.norm2(self.cnn2(self.pool1(x))))    
        x = tf.keras.activations.selu(self.norm3(self.cnn3(x)))
        x = tf.keras.activations.selu(self.norm4(self.cnn4(self.pool2(x))))
        x = tf.keras.activations.selu(self.norm5(self.cnn5(x)))
        x = tf.keras.activations.selu(self.norm6(self.cnn6(self.pool3(x))))    
        x = tf.keras.activations.selu(self.norm7(self.cnn7(self.pool4(x))))
        return self.final_dense(self.flatten(x))

In [0]:
class GeneratorModelOld(tf.keras.Model):

    def __init__(self):
        super(GeneratorModel, self).__init__(name='GeneratorModel')
        #filters = [256,256,256,256,256]
        filters = [64,64,64,64,64]
        kernel_sizes = [[1,1],[3,3],[5,5],[7,7],[9,9]]
        defilters = [64,3]
        self.cnn1 = layers.Conv2D(filters=filters[0],
                                  kernel_size=kernel_sizes[0],
                                  activation='tanh')
        self.cnn2 = layers.Conv2D(filters=filters[1],
                                  kernel_size=kernel_sizes[1],
                                  strides=1,           
                                  padding='same',
                                  activation='tanh')
        self.cnn3 = layers.Conv2D(filters=filters[2],
                                  kernel_size=kernel_sizes[2],
                                  strides=1,
                                  padding='same',
                                  activation='tanh')
        self.cnn4 = layers.Conv2D(filters=filters[3],
                                  kernel_size=kernel_sizes[3],
                                  strides=1,
                                  padding='same',
                                  activation='tanh')
        self.cnn5 = layers.Conv2D(filters=filters[4],
                                  kernel_size=kernel_sizes[4],
                                  strides=1,
                                  padding='same',
                                  activation='tanh')
        '''self.dense1 = layers.Dense(filters[0],activation='tanh')
        self.dense2 = layers.Dense(filters[1],activation='tanh')
        self.dense3 = layers.Dense(filters[2],activation='tanh')
        self.dense4 = layers.Dense(filters[3],activation='tanh')
        self.dense5 = layers.Dense(filters[4],activation='tanh')'''
        self.norm1 = layers.BatchNormalization()
        self.trans1 = layers.Conv2D(filters=defilters[0],
                                    kernel_size=[5,5],
                                    strides=1,
                                    padding='same',)
        self.norm2 = layers.BatchNormalization()
        self.trans2 = layers.Conv2D(filters=defilters[0],
                                    kernel_size=[5,5],
                                    strides=1,
                                    padding='same',)
        self.norm2 = layers.BatchNormalization()
        self.trans3 = layers.Conv2D(filters=defilters[0],
                                    kernel_size=[5,5],
                                    strides=1,
                                    padding='same',)
        self.norm2 = layers.BatchNormalization()
        self.transfinal = layers.Conv2D(filters=defilters[1],
                                        kernel_size=[3,3],
                                        strides=1,
                                        padding='same',)
      
      
    def call(self, inputs, training=False):
        x = inputs
        x1 = self.cnn1(x)
        x2 = self.cnn2(x)
        x3 = self.cnn3(x)
        x4 = self.cnn4(x)
        x5 = self.cnn5(x)
        x = x1 + x2 + x3 + x4 + x5
        #tf.broadcast_dynamic_shape(shape_x,shape_y)
        x = tf.keras.activations.selu(self.norm2(self.trans1(x)))
        return tf.keras.activations.tanh(self.transfinal(x))

In [0]:
class MaskGAN():
    def __init__(self, filePaths):
        self.tempdir = '/cnn/test/' + current_milli_time() + '/'
        self.img_shape = (224, 224, 3)
        self.real_images_paths, self.fake_images_paths = filePaths
        self.progress = []
        self.batch_size = 32
        self.last_image_w_h = (0,0)
        self.num_samples = 8
        self.border_size = 32
        
        inp = tf.keras.Input(shape=self.img_shape)
        discrim = self.createDiscriminator()
        validity = discrim(inp)
        self.discriminator = tf.keras.Model(inp, validity)
        self.discriminator.compile(optimizer=tf.keras.optimizers.Nadam(lr=2e-7),
                                   loss='binary_crossentropy',
                                   metrics=["accuracy"])
        
        self.generator = GeneratorModel()
        self.generator.compile(optimizer=tf.keras.optimizers.Nadam(lr=2e-5),
                               loss=self.generator_loss)
        z = tf.keras.Input(shape=self.img_shape)
        img = self.generator(z)

        self.discriminator.trainable = False
        valid = self.discriminator(img)
        self.combined = tf.keras.Model(z, valid)
        self.combined.compile(optimizer=tf.keras.optimizers.Nadam(lr=2e-5),
                              loss='binary_crossentropy')
    
    
    def generator_loss(self, labels, fake_images):
        l1_loss = tf.math.reduce_mean(tf.reshape(labels, [-1,]) - tf.reshape(fake_images, [-1,]))
        return -l1_loss
  
    def discriminator_loss(self, labels, preds):
        bce = tf.keras.losses.BinaryCrossentropy()
        return tf.reduce_mean(bce(labels, preds))
      
    def createDiscriminator(self):
        mobile_net = tf.keras.applications.MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
        mobile_net.trainable = True
        # Let's take a look to see how many layers are in the base model
        print("Number of layers in the base model: ", len(mobile_net.layers))

        # Fine tune from this layer onwards
        fine_tune_at = 65

        # Freeze all the layers before the `fine_tune_at` layer
        for layer in mobile_net.layers[:fine_tune_at]:
            layer.trainable =  False
          
        return tf.keras.Sequential([mobile_net,
                                    tf.keras.layers.GlobalAveragePooling2D(),
                                    tf.keras.layers.Dense(1, activation='sigmoid')])
      
      
    def trainDiscriminator(self, ds, ds_size=0, epochs=1, batch_size=32):
        if (ds_size==0):
            steps_per_epoch = 100
        else:
            steps_per_epoch=tf.ceil(ds_size/batch_size).numpy().astype('int32')
        self.discriminator.fit(ds, epochs=epochs, batch_size=batch_size, steps_per_epoch=steps_per_epoch)
        
    def trainGenerator(self, ds, ds_size=0, epochs=1, batch_size=32):
        if (ds_size==0):
            steps_per_epoch = 100
        else:
            steps_per_epoch=tf.ceil(ds_size/batch_size).numpy().astype('int32')
        self.generator.fit(ds,epochs=epochs, steps_per_epoch=steps_per_epoch)
        
        
    def train(self, epochs=1, batch_size=32,display_per_epoch=0):
          for i in range(epochs):
            print("Epoch =  %d" % i)
            shape = [None] + list(self.img_shape)
            balanced_gen = lambda: self.data_generator(batch_size)
            ds = tf.data.Dataset.from_generator(balanced_gen, (tf.float64, tf.float64), (tf.TensorShape(shape), tf.TensorShape([None])))
            ds.shuffle(batch_size*batch_size).prefetch(1)
            only_false_gen =  lambda: self.data_generator(batch_size,only_fake_images=True)
            only_false_ds = tf.data.Dataset.from_generator(only_false_gen, (tf.float64, tf.float64), (tf.TensorShape(shape), tf.TensorShape([None])))
            only_false_ds.shuffle(batch_size*batch_size).prefetch(1)
            only_true_gen =  lambda: self.data_generator(batch_size,only_true_images=True)
            only_true_ds = tf.data.Dataset.from_generator(only_true_gen, (tf.float64, tf.float64), (tf.TensorShape(shape), tf.TensorShape([None])))
            only_true_ds.shuffle(batch_size*batch_size).prefetch(1)
            gen_data = tf.data.Dataset.from_generator(lambda: self.data_generator_for_l1(batch_size), (tf.float64, tf.float64), (tf.TensorShape(shape), tf.TensorShape(shape)))
            gen_data.shuffle(batch_size*batch_size).prefetch(1)
            gen_images = self.generator.predict(only_false_ds,steps=1)
            gen_images = tf.cast(gen_images, tf.float64)
            offset_height = tf.cast(tf.floor((gen_images[0].shape.as_list()[0] - self.img_shape[0]) / 2), tf.int32)
            offset_width = tf.cast(tf.floor((gen_images[0].shape.as_list()[1] - self.img_shape[1]) / 2), tf.int32)
            gen_images = tf.map_fn(lambda image: tf.image.crop_to_bounding_box(image,offset_height,offset_width,self.img_shape[0],self.img_shape[1]), gen_images)
            print("Training Discriminator:")
            self.discriminator.fit(ds,steps_per_epoch=1,epochs=1)
            #self.discriminator.fit(only_static_gen,steps_per_epoch=1,epochs=1)
            self.discriminator.fit(only_true_ds,steps_per_epoch=1,epochs=1)
            self.discriminator.fit(gen_images,np.zeros(gen_images.shape[0]),steps_per_epoch=1,epochs=1)
            print("Training Generator:")
            self.combined.fit(only_false_ds,steps_per_epoch=1,epochs=1)
            #self.generator.fit(gen_data,steps_per_epoch=1,epochs=1)
            if (display_per_epoch>0 and i%display_per_epoch==0):
                for i in range(self.num_samples):
                    pred = self.generator.predict(self.data_generator(1, only_fake_images=True), steps=1)
                    self.upload_image(pred, self.tempdir + current_milli_time() + '.png')
                    time.sleep(.3)
    
    def data_generator_for_l1(self, batch_size):
        if (len(self.real_images_paths)<batch_size):
            real_batch_images = np.random.choice(self.real_images_paths, batch_size, replace=True)
        else:
            real_batch_images = np.random.choice(self.real_images_paths, batch_size, replace=False)
        if (len(self.fake_images_paths)<batch_size):
            fake_batch_images = np.random.choice(self.fake_images_paths,batch_size, replace=True)
        else:
            fake_batch_images = np.random.choice(self.fake_images_paths,batch_size, replace=False)
        real_batch_images = np.array(real_batch_images)
        real_image_ds = np.array(list(map(self.preprocess_image, real_batch_images)))
        fake_batch_images = np.array(fake_batch_images)
        fake_image_ds = np.array(list(map(self.preprocess_image, fake_batch_images)))
        yield fake_image_ds, real_image_ds
    
    def data_generator(self, batch_size, only_fake_images=False, only_true_images=False, only_static_images=False, staticProportion=0.2):
        if (only_true_images==False): num_real = batch_size//2
        else: 
            num_real = batch_size
            num_fake = 0
            num_static = 0
        if (only_fake_images==False): num_fake = batch_size-num_real
        else: 
            num_fake = batch_size
            num_real = 0
            num_static = 0
        if (only_static_images==True):
            num_static = batch_size
            num_fake = 0
            num_real = 0
        elif (only_true_images==False and only_fake_images==False):
            num_static = int(num_fake * staticProportion)
            
        final_ds_list = []
        final_label_list = []
        
        if (num_real>0):
            if (len(self.real_images_paths)<num_real):
                real_batch_images = np.random.choice(self.real_images_paths, num_real, replace=True)
            else:
                real_batch_images = np.random.choice(self.real_images_paths, num_real, replace=False)
            real_batch_images = np.array(real_batch_images)
            real_image_ds = np.concatenate(list(map(self.preprocess_image, real_batch_images)), axis=0)
            real_labels = np.array(np.ones(real_image_ds.shape[0]))
            #print(real_image_ds.shape)
            final_ds_list.append(real_image_ds)
            final_label_list.append(real_labels)

        if (num_fake>0):
            if (len(self.fake_images_paths)<num_fake-num_static):
                fake_batch_images = np.random.choice(self.fake_images_paths, num_fake-num_static, replace=True)
            else:
                fake_batch_images = np.random.choice(self.fake_images_paths, num_fake-num_static, replace=False)
            fake_batch_images = np.array(fake_batch_images)
            fake_image_ds = np.concatenate(list(map(self.preprocess_image, fake_batch_images)), axis=0)
            fake_labels = np.array(np.zeros(fake_image_ds.shape[0]))
            #print(fake_image_ds.shape)
            final_ds_list.append(fake_image_ds)
            final_label_list.append(fake_labels)
                
        if (num_static>0):
            static_images = [None for _q in range(num_static)]
            for i in range(len(static_images)):
                static_images[i] = self.get_static_image(self.img_shape[0],self.img_shape[1])
            static_images = np.stack(static_images)
            static_images = np.reshape(static_images, (-1,self.img_shape[0],self.img_shape[1],self.img_shape[2]))
            static_labels = np.array(np.zeros(num_static))
            final_ds_list.append(static_images)
            final_label_list.append(static_labels)       
        yield np.concatenate(final_ds_list, axis=0),np.concatenate(final_label_list, axis=0)
        
    def get_static_image(self, final_x, final_y):
        return np.random.random_sample((final_x, final_y, 3))
    
    def get_image(self, image_url, chunk_size=(224,224,3), dropbox=False, show=False):
        border_size=self.border_size
        if (dropbox):
            md, res = dbx.files_download(image_url)
            image = Image.open(res.raw)
        else: image = Image.open(image_url)
        self.last_image_w_h = width, height = image.size
        chunk_width,chunk_height,chunk_layers = chunk_size
        chunk_width,chunk_height = chunk_width-border_size*2,chunk_height-border_size*2
        if (height==None):
            print(image_url)
        
        chunks_wide = math.ceil(width/chunk_width)
        chunks_high = math.ceil(height/chunk_height)
        
        complete_width = chunks_wide * chunk_width
        complete_height = chunks_high * chunk_height
        
        padding = Image.new('RGB',(complete_width+border_size*2,complete_height+border_size*2))
        padding_size_w = math.ceil((complete_width-width)/2)
        padding_size_h = math.ceil((complete_height-height)/2)
        
        padding.paste(image, (padding_size_w+border_size,padding_size_h+border_size))
        if (show):
            display(padding)
            
        input_images = []
        
        for i in range(chunks_wide):
            for j in range(chunks_high):
                input_images.append(np.array(padding.crop((i*chunk_width,
                                                           j*chunk_height,
                                                           (i+1)*chunk_width+border_size*2,
                                                           (j+1)*chunk_height+border_size*2))))
        if (show):
            for img in input_images:
                display(Image.fromarray(img))
        
        return(np.stack(input_images).astype(np.float))
    
    def preprocess_image(self, file_address,show=False):
        images = self.get_image(file_address, chunk_size=self.img_shape,show=show)
        for i in range(images.shape[0]):
            images[i] = (2.0 * (images[i] / 255.0)) - 1
        return(images)
    
    #!!!!!!!! Can only recreate the last image processed by get_image !!!!!!!!
    def recreate_image(self, images, show=False):
        images = tf.map_fn(lambda img: tf.image.crop_to_bounding_box(img,
                                                      self.border_size,
                                                      self.border_size,
                                                      self.img_shape[0]-self.border_size,
                                                      self.img_shape[1]-self.border_size), images)
        original_width, original_height = self.last_image_w_h
        chunk_width,chunk_height,chunk_layers = self.img_shape
        chunk_width,chunk_height = chunk_width-self.border_size*2,chunk_height-self.border_size*2
        chunks_wide = math.ceil(original_width/chunk_width)
        chunks_high = math.ceil(original_height/chunk_height)
        backdrop = Image.new('RGB',(chunks_wide * chunk_width,chunks_high * chunk_height))
        input_images = []
        for i in range(images.shape[0]):
            img = ((1 + images[i]) / 2.0) * 255.0
            img = np.floor(img).astype(np.uint8)
            input_images.append(Image.fromarray(img))
        for p in range(chunks_wide):
            for q in range(chunks_high):
                backdrop.paste(input_images[(p*chunks_high)+q], (p*chunk_width,q*chunk_height))
        img = np.array(backdrop)
        starty = np.floor((img.shape[0] - original_height) / 2).astype(np.int32)
        startx = np.floor((img.shape[1] - original_width) / 2).astype(np.int32)
        img = img[starty:starty+original_height,startx:startx+original_width,:]
        img = Image.fromarray(img)
        if (show):
            display(img)
        return(img)
    
    def upload_image(self, images, image_dir):
        image = self.recreate_image(images)
        try:
            os.mkdir('/images/')
        except FileExistsError:
            pass
        image.save('/images/image.png', 'PNG')
        with open('/images/image.png', 'rb') as f:
            print(image_dir)
            dbx.files_upload(f.read(), image_dir, mute=True)
        os.remove('/images/image.png')

In [9]:
maskGan = MaskGAN((art_entries,flower_image_paths))

Instructions for updating:
Colocations handled automatically by placer.
Number of layers in the base model:  87


In [0]:
maskGan.train(epochs=1000, batch_size=3,display_per_epoch=100)

Epoch =  0
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Training Discriminator:
Instructions for updating:
Use tf.cast instead.
1/1 [==============================] - 0s 374ms/step - loss: 0.6751 - acc: 0.5625
Training Generator:
1/1 [==============================] - 11s 11s/step - loss: 0.7012
/cnn/test/1554194660/1554194688.png
/cnn/test/1554194660/1554194691.png
/cnn/test/1554194660/1554194693.png
/cnn/test/1554194660/1554194694.png
/cnn/test/1554194660/1554194696.png
/cnn/test/1554194660/1554194697.png
/cnn/test/1554194660/1554194699.png
/cnn/test/1554194660/1554194701.png
Epoch =  1
Training Discri

In [0]:
maskGan.train(epochs=1, batch_size=3, display_per_epoch=1)

In [0]:
choice = np.random.choice(art_entries)
images = maskGan.preprocess_image(choice,show=False)
image = maskGan.recreate_image(images,show=True)